In [1]:
import argparse
from Bio import SeqIO
import pandas as pd
# Running model
from sklearn.linear_model import LogisticRegression
# Loading model
import pickle
# Required for listing files
from os import listdir
from os.path import isfile, join
import os
# Loading/running model:
import tensorflow as tf

import scipy
from scipy import stats

import numpy as np

In [2]:
# Making the input robust to various 'boolean' inputs:
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

# Converting the ab1 to the fasta:
def abi_to_seq(input_ab1_file):
    # Opening the abi file:
    test_record = SeqIO.read(input_ab1_file, 'abi')
    # Reading in the sequence:
    letters = test_record.annotations['abif_raw']['PBAS1']
    return letters
###COMBINE THE FOLLOWING TWO FUNCTIONS??
# Listing all ab1 files in directory
def listing_ab1_files(input_dir):
    # Getting all of the ab1 files:
    onlyfiles = [f for f in listdir(input_dir) if
                 isfile(join(input_dir, f)) if '.ab1' in f]
    # Throwing on the directory to the front of the ab1 filenames:
    outputlfiles = ['%s' % input_dir + each_file for each_file in onlyfiles]
    return outputlfiles, onlyfiles

# Listing all ab1 files in directory
def listing_temp_files(input_dir):
    # Getting all of the temp files:
    onlyfiles = [f for f in listdir(input_dir) if
                 isfile(join(input_dir, f)) if 'temp.ab1.conv.' in f]
    # Throwing on the directory to the front of the ab1 filenames:
    return onlyfiles

# Converting sequence to fasta file:
def seq_to_fa(input_name, input_seq, sequence_name = None):
    # Getting the sequence name for the fasta:
    if sequence_name == None:
        sequence_name = input_name

    # Generating the fasta file:
    final_filename = input_name.rsplit('.', 1)[0] + '.fa'
    final_file = open(final_filename, 'w')
    final_file.write('> %s\n' % sequence_name)
    final_file.write(input_seq + '\n')
    final_file.close()
    return final_file

# Converting ab1 file to prediction input:
def abi_to_df(input_seqio_record):
    # Reading in the abi files:
    input_seqio_record = SeqIO.read(input_seqio_record, 'abi')

    # Getting the list of letters and their locations:
    locations = list(input_seqio_record.annotations['abif_raw']['PLOC1'])
    letters = list(input_seqio_record.annotations['abif_raw']['PBAS1'])

    # Converting to df:
    letter_loc_df = pd.DataFrame()
    letter_loc_df['Locations'] = locations
    letter_loc_df['Letters'] = letters

    # Different df with all the waveform data:
    peak_df = pd.DataFrame()
    peak_df['g_let'] = list(input_seqio_record.annotations['abif_raw']['DATA9'])
    peak_df['a_let'] = list(input_seqio_record.annotations['abif_raw']['DATA10'])
    peak_df['t_let'] = list(input_seqio_record.annotations['abif_raw']['DATA11'])
    peak_df['c_let'] = list(input_seqio_record.annotations['abif_raw']['DATA12'])

    # Making the indeces play nicely and deleting the other column:
    peak_df['index_plus_one'] = peak_df.index + 1
    peak_df.index = peak_df['index_plus_one']
    letter_loc_df.index = letter_loc_df['Locations']
    letter_loc_df.drop('Locations', inplace=True, axis=1)

    # combining the dfs:
    combined_df = letter_loc_df.join(peak_df, how='inner')
    return combined_df

# Adding the previous and the following base to the df:
def surrounding_bases(input_df):
    previous_letter_value_df = input_df.shift(1)
    previous_letter_value_df.dropna(inplace=True)
    previous_letter_value_df.rename({'a_let':'prev_a','c_let':'prev_c','t_let':'prev_t','g_let':'prev_g'}, inplace=True, axis=1)

    following_letter_value_df = input_df.shift(-1)
    following_letter_value_df.dropna(inplace=True)
    following_letter_value_df.rename({'a_let':'next_a','c_let':'next_c','t_let':'next_t','g_let':'next_g'}, inplace=True, axis=1)

    current_previous_following_df = pd.concat([input_df, previous_letter_value_df, following_letter_value_df], axis=1, join='inner')
    return current_previous_following_df

def ab1_to_predicted_sequence_nonorm(input_ab1_file, model, actual_ab1=True, denormalize=False):
    # Loading in and parsing input df:
    if actual_ab1 == True:
        test_df = abi_to_df(input_ab1_file)
    else:
        test_df = input_ab1_file
    test_letter_value_df = test_df[['a_let', 'c_let', 't_let', 'g_let']]
    # Rerun the nucleotide model on normalized values, but until then:
    if denormalize == True:
        test_letter_value_df = test_letter_value_df * 1000
    test_full_info_df = surrounding_bases(test_letter_value_df)

    # Using model to predict sequence:
    predicted_probs_df = pd.DataFrame(model.predict(X=test_full_info_df),
                                      columns=['Prediction'])

    # Acquiring and returning sequence:
    sequence = ''.join(list(predicted_probs_df['Prediction']))
    return sequence

# This combines a peak df with a full record
def peak_calling_df(input_df, input_seqio_record):
    input_df['peak_no_peak'] = [1] * input_df.shape[0]
    input_df.index = input_df.index + 1####MAYBE KEEP THIS IN? MAYBE REMOVE IT?
    first_val = input_df.index[0] - 5
    last_val = input_df.index[-1] + 5
    removed_df = input_df[['peak_no_peak']]
    # Different df with all the waveform data:
    peak_val = pd.DataFrame()
    peak_val['g_let'] = list(input_seqio_record.annotations['abif_raw']['DATA9'])
    peak_val['a_let'] = list(input_seqio_record.annotations['abif_raw']['DATA10'])
    peak_val['t_let'] = list(input_seqio_record.annotations['abif_raw']['DATA11'])
    peak_val['c_let'] = list(input_seqio_record.annotations['abif_raw']['DATA12'])

    peak_val = peak_val.loc[first_val:last_val]
    fin_df = removed_df.merge(peak_val, how='outer', left_index=True, right_index=True)
    zero = fin_df[fin_df['peak_no_peak'] !=1]
    zero['peak_no_peak'] = [0] * zero.shape[0]
    nonzero = fin_df[fin_df['peak_no_peak'] ==1]
    fin_df = zero.append(nonzero)
    fin_df.sort_index(inplace=True)
    return fin_df

def slope(inp_df):
    only_letters = inp_df[['g_let', 'a_let', 't_let', 'c_let']]
    slope_before = only_letters.diff(1, axis=0)
    slope_before.columns = ['slope_g_after', 'slope_a_after', 'slope_t_after', 'slope_c_after']
    slope_after = only_letters.diff(-1, axis=0)
    slope_after.columns = ['slope_g_before', 'slope_a_before', 'slope_t_before', 'slope_c_before']

    final = only_letters.join(slope_before)
    final = final.join(slope_after)
    final = final.join(inp_df[['peak_no_peak']])
    return final

def normalizing(inp_df):
    all_peak_places = inp_df[inp_df['peak_no_peak'] == 1]
    all_peak_places_vals = all_peak_places[['g_let', 'a_let', 't_let', 'c_let']]
    all_max_peaks = list(all_peak_places_vals.max(axis=1))
    trimmed_mean = scipy.stats.trim_mean(all_max_peaks, proportiontocut=0.1)
    inp_df = inp_df / trimmed_mean
    inp_df['peak_no_peak'] = inp_df['peak_no_peak'] * trimmed_mean
    inp_df['peak_no_peak'] = inp_df['peak_no_peak'].astype(int)
    return inp_df

def reshaping_the_df(inp_df, first_dim, second_dim, third_dim):
    y_val_train = np.array(inp_df['peak_no_peak'])
    inp_df = inp_df.iloc[:,:-1]
    x_val_train = inp_df.values.reshape((first_dim,second_dim,third_dim))
    return x_val_train, y_val_train


# The function to swap out current peak for a better one
def finding_taller_peak(the_input_df):
    # Getting all of the peaks:
    peaks = the_input_df[the_input_df['peak_no_peak'] == 1]
    # Getting the indeces of all of the peaks:
    peak_indeces = list(peaks.index)
    # Getting the surrounding four rows:
    before_peaks = [item - 4 for item in peak_indeces]
    after_peaks = [item + 4 for item in peak_indeces]
    # Going through each peak and respective rows:
    for idx, item in enumerate(before_peaks):
#         Making the smaller df:
        temp_df = the_input_df.loc[item:after_peaks[idx]]
        current_peak_loc = temp_df[temp_df['peak_no_peak'] == 1].index.tolist()[0]
        just_letts = temp_df[['g_let','a_let','t_let','c_let']]
#         Getting the max value of the dataframe:
        max_val = max(just_letts.max(axis=1))
#         Getting the index of the max value:
        max_idx = int(temp_df[temp_df.values == max_val].index.tolist()[0])
        max_letter = just_letts.loc[max_idx].idxmax(axis=1)

#         Acquiring the slopes:
        before_slopes = pd.DataFrame(temp_df.loc[max_idx][['slope_g_before','slope_a_before','slope_t_before','slope_c_before']]).T
        before_slopes.columns = ['g_let','a_let','t_let','c_let']
        after_slopes = pd.DataFrame(temp_df.loc[max_idx][['slope_g_after','slope_g_after','slope_g_after','slope_g_after']]).T
        after_slopes.columns = ['g_let','a_let','t_let','c_let']

#         If they're both positive:
        val_1 = before_slopes[[max_letter]].values >= 0
        val_1 = val_1[0][0]
        val_2 = after_slopes[[max_letter]].values >= 0
        val_2 = val_2[0][0]
        if val_1 and val_2:
            the_input_df['peak_no_peak'][current_peak_loc] = 0
            the_input_df['peak_no_peak'][max_idx] = 1     

    return the_input_df

# Getting the called nucleotide from the cleaned up peak:
def get_nuc_with_clean(input_index, input_df, df_or_val='val'):
    if df_or_val == 'val':
        og_nuc = input_df.iloc[input_df.index.get_loc(input_index,method='nearest')]['letter_vals']
    else:
        og_nuc = input_df.iloc[input_df.index.get_loc(input_index,method='nearest')]
    return og_nuc

def normalizing_nucleotides(inp_df):
    all_peak_places_vals = inp_df[['g_let', 'a_let', 't_let', 'c_let']]
    all_max_peaks = list(all_peak_places_vals.max(axis=1))
    trimmed_mean = scipy.stats.trim_mean(all_max_peaks, proportiontocut=0.1)
    all_peak_places_vals = all_peak_places_vals / trimmed_mean
    return all_peak_places_vals


def fixing_peaks_and_normalizing(input_ab1_file):
    ab1_record = SeqIO.read(input_ab1_file, 'abi')
    nucleotide_record = abi_to_df(input_ab1_file)
    fin_training = peak_calling_df(nucleotide_record, ab1_record)
    fin_training = normalizing(fin_training)
    fin_training = slope(fin_training)
    finding_taller_peak(fin_training)
    final_df = fin_training[fin_training['peak_no_peak'] == 1]
    final_df.reset_index(inplace=True)
    final_df.drop('index', axis=1, inplace=True)
    return final_df
        
        

In [77]:
! ls

README.md
ab1_dir
ab1_dir.fa
data
generating_and_evaluating_model.ipynb
log_reg_default_million.sav
log_reg_default_million_normed_fixed_peaks_tuned_parameters.sav
model.h5
sang.py
troubleshooting.ipynb


In [122]:
updated_model = pickle.load(open('log_reg_default_million_normed_fixed_peaks_tuned_parameters.sav', 'rb'))

In [123]:
old_model = pickle.load(open('log_reg_default_million.sav', 'rb'))

In [145]:
input_ab1_file = ('./ab1_dir/other/tsh_7_kb_9_17_19.custom3.ab1')

In [151]:
first_round_df = fixing_peaks_and_normalizing(input_ab1_file)
updated_predicted = ab1_to_predicted_sequence_nonorm(first_round_df, updated_model, actual_ab1=False)

/Users/alexandredaly/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alexandredaly/anaconda/envs/python3/lib/python3.5/site-packages/scipy/stats/stats.py:2825: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/alexandredaly/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [152]:
updated_predicted

'ATAAGCACCAATTTATTATGCACACCTAGCTGAAAACCCAATTTTAGCTACAGCTTGGAGGCAAATAGCTGTCTTAGTCAAATAGCTATCAGTCCTTATCAATCATGAGCAGCCGACTGGTCAGAACACACTCATGTAAGCTCCACGTCTGGACAGCACCCATCAGACTGTTTCTGCGCCTTGCTCTGCTCTCTGTCCACAGATGCTCTGGCGTCTGGCGTCGCTGAGCTCCCCTTAGCTTTGTTTTCACAGGGCTGTCTGACTTGTGGATCTTTCTTCCTTCGAACGCACTCAAATGAGTGTCCTGCAAGGAATGCCAGCATCCATCCTGCAGAGAGAAGACATAGTCTCTCACTAACATACATAAAGAAGGTAAGAGTAAGTGAGTACAGACAAGATTGACTTTCACCACTACCAATAAATATAATTAATTTCACTCTCTATCAAATGCATGGCGTGCGATCTATCTAACACACACACAAATACCACCCATCCGACAAACAATGAATAAATAGACAAAGAATGGCTTGCATGATGGTTGCTGACAGAAATCATCAATGATGAACAAGAGTGACCTGACAGAAAACATACAAGGAATGAATGCCTGGGTGTGCGTTCAGATGGAGCGAACTAAATACTCCCGGACAGGAAGGAAGAAAGTCACTCGCTAGACAAACTGCATGTTGTCTGACTCCACCCGAAGGAGACAAAGAGTGTATCTTGCAAATAAAATCTAAAAGATAAGTGTTGGTTGGGCTAAGCGAATGCATAACAGTACAGATTACAAGTCATGGAAGGCAGGACCGAAACTTCCTTTTCTTTCCCATTTCCCATTAAATTTTAATTTTTGGAAATANATATTTTACNTAAAANCTCCAAGTTGCAAATGCCCAATAATTATAACCTTTACCCTATACCCATCCGCCCTGGCCCCCGCTTCCCCAAACCCATTTNCATTTTCTTTGGCCTTGGGCCATTTGCCCTTGCCACTGGGGGCNT

In [153]:
old_predicted = ab1_to_predicted_sequence_nonorm(input_ab1_file, old_model)

In [154]:
old_predicted

'AGACGCACCAATTTATTATGCACACCTAGCTGAAAACCCAATTTTAGCTACAGTTTGGAGGCAAATAGCTGTCTTAGTCAAATAGCTATCAGTCCTTATCAATCATGAGCAGCCGACTGGTCAGAACACACTCATGTAAGCTCCACGTCTGGACAGCACCCATCAGACTGTTTCTGCGCCTTGCTCTGCTCTCTGTCCACAGATGCTCTGGCGTCTGGCGTCGCTGAGCTCCCCTTAGCTTTGTTTTCACAGGGCTGTCTGACTTGTGGATCTTTCTTCCTTCGAACGCACTCAAATGAGTGTGCTGCAAGGATTGCCAGCATCCATCATGCAGGGAGCATACTTAGTCTCTCACTAACATACATAATGAAGGTAAGAGTAAGTGAGTACAAACAGGATTGACAATAACCACCACCAATAACTATAATTTTACTCACTCTCTACCAAATGCATGGGGTGCGATCTATCTATCACTCACACAACTCCCACTCATCCAACAAACCTTGCATAATCAGACAAAGATTGGCTAGCGTGATGGTTGCTGACAGAAATCATGAATGATGAACAAGAGTGACCTGCCATAAAACATGCAAGGAATGACTGCCTGGGTGTGCGTTCAGATGGAGCAAACTAATCACTCCCGGACAGGAAGGAAGAAACTCACTCGCTAGACAAACTGCATGTTGTCTGACTCCACCCGAAGGAGACAAAGTGTGTATCTTGCAAATAAAATCTAGAAGATAAGTGTTGGTTGGGCTAANCGNATGCATAACAGTACAGANTACAANTCNTGGAANGCANGACCGAANCTTNCTTTTCTTTCCCATNTCCCATTNATTNTTANTATTTNGATNANNATATTTTACNTNNAANNNNCAANNTGNAAATGCCNAAAAATTTTAANCTTTCACCTCTNCCCATNNGCNCTGGCNCCNGCTTCCCCATACCCANTTNCTTTTTCTGTGGCCTTGGGCCATTNGCCNTTGCCACTGGGGGCNC

In [155]:
input_ab1_file

'./ab1_dir/other/tsh_7_kb_9_17_19.custom3.ab1'

In [159]:
import os

In [160]:
# Making the training df larger:
fixed_path = './data/fixed_peaks/'
unfixed_path = './data/unfixed_peaks/'


Quickly changing the names of the files:

In [170]:
df_files

['ab1_file_2158.csv',
 'ab1_file_4058.csv',
 'ab1_file_3669.csv',
 'ab1_file_2173.csv',
 'ab1_file_1863.csv',
 'ab1_file_605.csv',
 'ab1_file_1527.csv',
 'ab1_file_2348.csv',
 'ab1_file_562.csv',
 'ab1_file_1939.csv',
 'ab1_file_3915.csv',
 'ab1_file_4667.csv',
 'ab1_file_3646.csv',
 'ab1_file_2073.csv',
 'ab1_file_2926.csv',
 'ab1_file_2113.csv',
 'ab1_file_1630.csv',
 'ab1_file_2462.csv',
 'ab1_file_1023.csv',
 'ab1_file_2313.csv',
 'ab1_file_2707.csv',
 'ab1_file_1435.csv',
 'ab1_file_3368.csv',
 'ab1_file_1360.csv',
 'ab1_file_2641.csv',
 'ab1_file_1957.csv',
 'ab1_file_1968.csv',
 'ab1_file_1428.csv',
 'ab1_file_2928.csv',
 'ab1_file_4418.csv',
 'ab1_file_286.csv',
 'ab1_file_2081.csv',
 'ab1_file_4285.csv',
 'ab1_file_141.csv',
 'ab1_file_3110.csv',
 'ab1_file_2973.csv',
 'ab1_file_3098.csv',
 'ab1_file_3265.csv',
 'ab1_file_3477.csv',
 'ab1_file_4863.csv',
 'ab1_file_2962.csv',
 'ab1_file_2976.csv',
 'ab1_file_4453.csv',
 'ab1_file_1475.csv',
 'ab1_file_4727.csv',
 'ab1_file_409

In [5]:
nucleotide = './data/ab1_for_evaluation/'
df_files = [f for f in listdir(nucleotide) if isfile(join(nucleotide, f))]

In [6]:
random_names = np.random.choice(len(df_files)*100,len(df_files), replace=False)
for idx, item in enumerate(df_files):
    new_name = 'ab1_file_nucleotide_%s.ab1' % str(random_names[idx])
    # Renaming the fixed
    os.rename('%s%s' % (nucleotide, item), '%s%s' % (nucleotide, new_name))

In [165]:
for idx, item in enumerate(df_files):
    try:
        new_name = 'ab1_file_%s.csv' % str(random_names[idx])
        os.rename('%s%s' % (nucleotide, item), '%s%s' % (nucleotide, new_name))
    except:
        continue

In [162]:
random_names

array([3368, 2926, 4418, 2973, 3669, 1630, 2641, 4667, 3915, 2928, 2962,
       2158, 3265, 4450, 1475, 3477,  141, 2113, 1023, 2707,  605, 1968,
       1360, 1435, 1957, 3935, 4453, 2081, 2462, 1939, 2348, 2073, 4861,
        562, 2173, 3646, 4058, 2976, 4863, 1527, 3098, 4285, 2313, 4727,
       3110, 4096, 1863,  286, 4929, 1428])